In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf


print(tf.test.is_gpu_available())
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\witlab\anaconda3\envs\

True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14187732203381456133
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 78607156
locality {
  bus_id: 1
}
incarnation: 15757150696017018669
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten

def unet_model(input_shape):
    model = Sequential()

    # Encoder
    model.add(Conv2D(64, (3,3), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D())

    model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D())

    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D())

    model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D())

    # Bottleneck
    model.add(Conv2D(1024, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(1024, (3,3), activation='relu', padding='same'))

    # Decoder
    model.add(UpSampling2D())
    model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3,3), activation='relu', padding='same'))

    model.add(UpSampling2D())
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

    model.add(UpSampling2D())
    model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3,3), activation='relu', padding='same'))

    model.add(UpSampling2D())
    model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(Flatten())

    # Output layer
#     model.add(Conv2D(12, 1, activation=None))  # 12 output channels for 12 illuminance values
    model.add(Dense(12))
    return model


In [4]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping_cb = EarlyStopping(patience =20, monitor = 'val_loss')
checkpoint_cb = ModelCheckpoint("model/unet_best_model_sensor_add_img_no_cut_20240313.h5", monitor='val_loss', save_best_only=True, mode='min')

In [5]:
# 이미지 불러오기

image_path=r'image\auto_intg\sensor_add\preprocess_no_cut\train_image_20240313'
image_files = [f for f in os.listdir(image_path) if f.endswith('.jpg')]

images = []
for file in image_files:
    img = cv2.imread(os.path.join(image_path, file))
    if img is None:
        print(f"{file} 읽을 수 없음")
        continue

    img = img / 255.0  # 0~1로 정규화
    images.append(img)

images = np.array(images)
print(images.shape)

(2506, 224, 224, 3)


In [6]:
# 데이터 불러오기
import pandas as pd

excel_path = 'data/train_data_20240307-20240312_20240313.xlsx'
df = pd.read_excel(excel_path, engine='openpyxl')
illum_columns = [f'illum_{i}' for i in range(1, 13)]
target_data = df[illum_columns].to_numpy()
target_data[target_data < 0] = 0  # Clip negative values to 0

print(target_data.shape)

(2506, 12)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, target_data, test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

Training data shape: (2004, 224, 224, 3)
Testing data shape: (502, 224, 224, 3)
Training target shape: (2004, 12)
Testing target shape: (502, 12)


In [8]:
# 모델 학습

input_shape = (224, 224, 3)
model = unet_model(input_shape)


model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, epochs=500, batch_size=2,validation_data=(X_test, y_test), callbacks=[early_stopping_cb,checkpoint_cb])

# 모델 평가
test_loss = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)       295168    
__________

KeyboardInterrupt: 